<div style="float:left">
    <h1 style="width:600px">Workshop 6: Spatial Clustering</h1>
    <h3 style="width:600px">CASA0006: Data Science for Spatial Systems</h3>
    <h3 style="width:600px">Author: Huanfa Chen</h3>
</div>
<div style="float:right"><img width="100" src="https://github.com/jreades/i2p/raw/master/img/casa_logo.jpg" /></div>

## Introduction

In this workshop, you will practice the following skills:

1. Using various clustering methods (e.g. kmeans, DBSCAN, hierarchical, max-p) to analyse the housing price and population data in London; 
2. Describing and analysing the clustering results;
3. Selecting the key parameters for kmeans clustering and hierarchical clustering.

In [1]:
import datetime
now = datetime.datetime.now()
print("Last executed: " + now.strftime("%Y-%m-%d %H:%M:%S"))

Last executed: 2022-02-11 12:49:02


## Preparation
We will download the GIS and London housing data from the London DataStore. Here, we will practice with Linux/Windows commands to automatically download the data. If you are not familiar with these commands, you can download the data by visiting the URL in the browser and then saving the dataset to a folder. 

*Hint: make sure that you put the dataset in the same folder as this Python notebook.*

In [ ]:
import pysal as ps
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px

from math import ceil

from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, DBSCAN, OPTICS, AgglomerativeClustering
from esda.adbscan import ADBSCAN

from scipy.cluster.hierarchy import dendrogram

import spopt
from spopt.region import MaxPHeuristic as MaxP
import matplotlib.pyplot as plt

import libpysal
import warnings

Next we will download the data. The ```wget``` command will work for Mac and Linux, but not for Windows. If you are using Windows and get the following error, don't panic - just open the url on your browser and save the zip file to an appropriate folder.

```
'wget' is not recognized as an internal or external command, operable program or batch file.
```

In [ ]:
# note that this dataset contains all 7201 MSOAs in England. This dataset is used as it contains the high resolution boundary and neighbour topology of MSOAs 
url = 'https://github.com/jreades/fsds/raw/master/data/src/Middle_Layer_Super_Output_Areas__December_2011__EW_BGC_V2-shp.zip'

! wget $url

In [ ]:
gdf = gpd.read_file(f"zip://Middle_Layer_Super_Output_Areas__December_2011__EW_BGC_V2-shp.zip!Middle_Layer_Super_Output_Areas__December_2011__EW_BGC_V2.shp")
gdf.info()

Next, we will download the house price in a csv file from the [London DataStore](data.london.gov.uk). 

In [ ]:
df = pd.read_csv('https://data.london.gov.uk/download/average-house-prices/bdf8eee7-41e1-4d24-90ce-93fe5cf040ae/land-registry-house-prices-MSOA.csv', na_values=[':'], low_memory=False)

In [ ]:
print(df.shape)
measure = 'Mean'
year = 2017
df = df[(df.Year==f'Year ending Dec {year}') & (df.Measure==measure)].copy().reset_index()
df.drop(index=df[df.Value.isna()].index, inplace=True)
print(df.shape)

In [ ]:
df['Measure'] = df.Measure.astype('category')
df['Value'] = df.Value.astype('int')
df.drop(columns=['index'], inplace=True)

In [ ]:
ppd = pd.merge(gdf, df, left_on='MSOA11CD', right_on='Code', how='inner').reset_index()
ppd.shape

How many MSOAs are there in the **ppd** dataset? In other words, how many MSOAs are there in London?

In [ ]:
# ppd.shape[??]
ppd.shape[0]

In [ ]:
ppd[ppd.Value.isin([ppd.Value.max(), ppd.Value.min()])][['index','MSOA11CD','MSOA11NM','Value']]

In [ ]:
def replace_legend_items(legend, mapping):
    for txt in legend.texts:
        for k,v in mapping.items():
            if txt.get_text() == str(k):
                txt.set_text(v)

In [ ]:
tfont = {'fontname':'Liberation Sans Narrow', 'horizontalalignment':'left'}

Next, we will enrich the MSOA with population data. We will download the MSOA census data and merge with the ppd data.

In [ ]:
# note that this dataset contains all 7201 MSOAs in England. This dataset is used as it contains the high resolution boundary and neighbour topology of MSOAs 
url = 'https://data.london.gov.uk/download/statistical-gis-boundary-files-london/9ba8c833-6370-4b11-abdc-314aa020d5e0/statistical-gis-boundaries-london.zip'

! wget $url

In [ ]:
gdf_london_msoa_census = gpd.read_file(f"zip://statistical-gis-boundaries-london.zip!statistical-gis-boundaries-london/ESRI/MSOA_2011_London_gen_MHW.shp")

In [ ]:
gdf_london_msoa_census.columns

In [ ]:
gdf_london_msoa_census = gdf_london_msoa_census[['MSOA11CD', 'USUALRES', 'HHOLDRES', 'POPDEN']]

In [ ]:
ppd = pd.merge(ppd, gdf_london_msoa_census, left_on='MSOA11CD', right_on='MSOA11CD', how='inner').reset_index()
ppd.shape

### Raw Distribution

What is the housing price distribution in London? We can use some plots and maps to showcase the distribution.

In [ ]:
f = plt.figure()
f.set_size_inches(12,7)

ax1 = f.add_subplot()
ppd.plot(column='Value', legend=True, cmap='plasma', figsize=(12,6), ax=ax1)

f.subplots_adjust(top=0.92)
f.suptitle(f"{measure} House Prices (Raw)", x=0.025, size=24, **tfont);
plt.savefig('Cluster-House-Prices-Raw.png', dpi=150)

## Variable Selection and Standardisation

We will use the ```RobustScaler``` class from the ```sklearn``` package for standardising the dataset. Alternatively, you can use the ```MinMaxScaler``` class.

The RobustScaler is robust to outlier. It removes the median and scales the data according to the quantile range (defaults to between 25 quantile and 75 quantile, also called the Interquartile Range or IQR).

If you are not sure about these two classes, have a quick read of the documentation.

In [ ]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
rs = RobustScaler(quantile_range=(10.0, 90.0))

In [ ]:
ppd.head()

Here we focus on three variables: 

* ```POPDEN```: population density
* ```Value```: the mean house prices
* ```USUALRES```: number of usual residents
* ```HHOLDRES```: number of household residents

On the other hand, we should include the ```MSOA11CD``` variable as the index of MSOAs. If you want to know more about the ```index``` in a Pandas DataFrame (or Series), please read the documentation.

In [ ]:
raw = ppd[['MSOA11CD','USUALRES','HHOLDRES', 'POPDEN','Value']].set_index('MSOA11CD').copy()
raw.head()

Plot a histogram of the mean house prices with 150 bins. Complete the code.

In [ ]:
raw.???.plot.hist(???=150)

In [ ]:
normed = raw.copy()
for c in raw.columns.values:
    normed[c] = rs.fit_transform(raw[c].values.reshape(-1,1))
    print("The range of {} is [{}, {}]".format(c, normed[c].min(), normed[c].max()))
normed.head()

One important note on `scikit-learn`: the `scikit-learn` methods operate in a very similar fashion, regardless of the approach you are using. It uses the `.fit()` or `.fit_transform()` function execute the model and return results. Each model will take different arguments, but the same approach is used every time.

Make a scatter plot between **Value** (on the y axis) and **POPDEN** (on the x axis).

Think about this question: is it that the higher population density, the higher average house price?

In [ ]:
normed.plot.scatter(x= ???, y=???)

## Functions

A function for mapping the clustering results:

In [ ]:
def mapping_clusters(labels_cluster):
    ppd['cluster_nm'] = labels_cluster
    ppd.plot(column='cluster_nm', categorical=True, legend=True, figsize=(12,8), cmap='Paired');

A function for the radar plot of the cluster centroids:

In [ ]:
# adapted from this tutorial: https://towardsdatascience.com/how-to-make-stunning-radar-charts-with-python-implemented-in-matplotlib-and-plotly-91e21801d8ca
def radar_plot_cluster_centroids(df_cluster_centroid):
    # parameters
    # df_cluster_centroid: a dataframe with rows representing a cluster centroid and columns representing variables
    
    # add an additional element to both categories and restaurants that’s identical to the first item
    # manually 'close' the line
    categories = df_cluster_centroid.columns.values.tolist()
    categories = [*categories, categories[0]]
    
    label_loc = np.linspace(start=0, stop=2 * np.pi, num=len(categories))
    
    plt.figure(figsize=(12, 8))
    plt.subplot(polar=True)
    for index, row in df_cluster_centroid.iterrows():
        centroid = row.tolist()
        centroid = [*centroid, centroid[0]]
        label = "Cluster {}".format(index)
        plt.plot(label_loc, centroid, label=label)
    plt.title('Cluster centroid comparison', size=20, y=1.05)
    lines, labels = plt.thetagrids(np.degrees(label_loc), labels=categories)
    plt.legend()
    plt.show()

## DBSCAN

So, let's run the DBSCAN clustering method. According to the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html), the model can take a number of parameters: the `eps` distance and `min_samples` group size attributes are fundamental in creating the clusters. I've suggested some values below, but we'll try amending those later.

### First DBSCAN

In [ ]:
minPts = 5 # we set minPts as normed.shape[1] + 1 
epsilon = 0.2
dbsc = DBSCAN(eps=epsilon, min_samples=minPts)
dbsc.fit(normed)

# We now have our DBSCAN object created, and we can extract the groups it has identified. We do this using the `.labels_` method.
cluster_nm = dbsc.labels_

mapping_clusters(cluster_nm)

Some might think there are five clusters corresponding to five different colours - don't presume! If you understand DBSCAN, it classifies the points that are far away from others as outliers.

According to the DBSCAN [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html), it says:

```
Noisy samples are given the label -1.
```

The MSOAs in blue (labelled -1) are actually noise samples. So the above map show there are FOUR clusters.

How many MSOAs are there in each cluster?

We can summarise the number of MSOAs in each cluster, using the ```value_counts``` function.

In [ ]:
pd.Series(dbsc.labels_).value_counts()

You can also visualise the cluster centroid using the radar plot.

In [ ]:
df_dbscan = normed.copy()
df_dbscan['cluster'] = dbsc.labels_
df_dbscan_centroid = df_dbscan.groupby('cluster').mean()
# drop the outlier
df_dbscan_centroid.drop(-1, inplace=True)
# df_dbscan_centroid.reset_index()
radar_plot_cluster_centroids(df_dbscan_centroid)

**So what do you think of the results? Do the clusters look useful or realistic?**

The next thing to do is measure the formation of the clusters. We can do this through a range of measures - described in detail here [here](http://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation). Most of these measures, however, require a groundtruth relating to how a cluster should look (e.g. help us supervise the creation of the clusters. Unfortunately, we do not have such groundtruth.

The only measure that can help is the Silhouette Score, which calculates how close points are on average to points their clustered with, relative to points they are not clustered with.

The `scikit-learn` algorithm ([docs](http://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient)) for Silhouette Score simply takes the data and the generated labels. A score closer to one indicates strong clustering, negative scores indicate poor clustering.

**Run the code below to first load the library, then extract the Silhouette Score for the clusters created above.**

In [ ]:
from sklearn import metrics
metrics.silhouette_score(normed, dbsc.labels_)

The best and worst value of the silhouette score is 1 and -1, respectively. Values near 0 indicate overlapping clusters. Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar.

Now you can change the value of ```eps``` and ```minPts``` and see whether the clustering quality has been improved. You can always use the Silhouette score to measure the clustering quality.

## Kmeans

In this part, we will use the K-Means method on the same dataset. 

### First Kmeans

First, we try k=4, as the result of DBSCAN indicates that there might be 4 clusters.

In [ ]:
from sklearn.cluster import KMeans

The workflow of K-Means is very similar with that of DBSCAN.

In [ ]:
k_cluster = 4
random_seed = 1
kmeans_method = KMeans(n_clusters=k_cluster,random_state=random_seed)
kmeans_method.fit(normed)

You can visualise the clusters on the map.

In [ ]:
mapping_clusters(kmeans_method.labels_);

You can also visualise the cluster centroid using the radar plot.

In [ ]:
df_cluster_centroid = pd.DataFrame(kmeans_method.cluster_centers_, columns=normed.columns)
radar_plot_cluster_centroids(df_cluster_centroid)

### Randomness of K-Means

K-Means involves randomness. The parameter of **random_state** controls the initialisation of the cluster centroids. If you change the value of random_state, the initial centroids will be different and you will probably get a different clustering. If you don't specify the random_state in the code, then every time you execute your code a new random value is generated and the train and test datasets would have different values each time.

The reason for setting the random_state here is to reproduce the result. But in real-world projects, you are not encouraged to manipulate the result via testing with different random_state values.

A common practice with K-Means (and other algorithms involving randomness) is to run the algorithm many times using different random seeds and then choose the best result, such as the result with the lowest SSE. In the sklearn K-Means function, the parameter of **n_init** (default at 10) specifies the number of times the K-Means algorithm will be run with different centroid seeds. The final result will be the best output of **n_init** runs in terms of inertia (aka, SSE).

Read more about K-Means in the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

### Finding the 'right' number of clusters

Next, you will try the Elbow method to choose the k value. The idea behind the Elbow method is to identify the value of k where the distortion begins to decrease most rapidly. At this value, if you continue to increase the k, you will not get much decrease on the SSE.

In [ ]:
# calculate SSE for a range of number of cluster
list_SSE = []
min_k = 1
max_k = 10
range_k = range(min_k, max_k+1)
for i in range_k:
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(normed)
    # inertia is a concept from physics. Roughly it means SSE of clustering.
    list_SSE.append(km.inertia_)

# plot
plt.plot(range_k, list_SSE, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

From the SSE plot, you can see that **three** might be the optimal k value. Can you re-run the K-Means function with k = 3?

In [ ]:
k_cluster = ??
random_seed = ??
kmeans_method = KMeans(??=??,??=??)
kmeans_method.fit(??)

# plotting
mapping_clusters(???.labels_);

## Hierarchical clustering

We will test the Agglomerative Clustering (one type of hierarchical clustering) for this dataset. 

First, we will generate and plot the dendrogram from the clustering process. This shows the hierarchy of the clusters.

Given the hundreds of data points, the whole dendrogram starting from the individual level would be complicated and challenging to visualise. Therefore, we will show the top three levels of the dendrogram. 

The code of **plot_dendrogram** is largely based on this [tutorial](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py). More details about the dendrogram function from scipy is [here](https://docs.scipy.org/doc/scipy-1.8.0/html-scipyorg/reference/generated/scipy.cluster.hierarchy.dendrogram.html?highlight=dendrogram#scipy.cluster.hierarchy.dendrogram).

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, leaf_rotation=90., **kwargs)

In [ ]:
agg_cluster = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(normed)
ax = plt.gca()
plt.title("Hierarchical Clustering Dendrogram")
# plot the top three levels of the dendrogram
plot_dendrogram(agg_cluster, truncate_mode="level", p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.ylabel('distance')
# note that these two red dashed lines are drawn manually, as we see that the gap between the two lines is larger than other gaps
plt.hlines(10.7, ax.get_xlim()[0], ax.get_xlim()[1], linestyle='dashed', color='r')
plt.hlines(14.8, ax.get_xlim()[0], ax.get_xlim()[1], linestyle='dashed', color='r')
plt.show()

The next task is to look at this dendrogram and understand the impact of different distance cut-offs on the clusters. 

This dendrogram shows that there is a big gap between the distance of 10.7 to the next merge (at the distance of around 14.8). The distance gap is highlighted by two red dashed lines. Such distance gap in the dendrogram is pretty interesting, as it indicates that the next merge possibly shouldn't happen. In other words, it is likely the things that are merged here really don't belong to the same cluster, telling that four clusters are appropriate here.

In [ ]:
agg_cluster = AgglomerativeClustering(distance_threshold=None, n_clusters=4).fit(normed)
mapping_clusters(agg_cluster.labels_)

In [ ]:
pd.Series(agg_cluster.???).value_counts()

You can find the mapping from the four clusters to the clusters in the above dendrogram, using the number of points in each cluster.

In the above dendrogram with the cutoff distance of around 12.5, the four clusters from left and right corresponds to Cluster 3, 2, 1, and 0 in the map.

## Regionalisation: max-p problem

The clusters generated by DBSCAN/Kmeans/Hierarchical clustering methods are not spatially contiguous. In other words, a cluster is very likely to contain MSOAs that are not spatially connected.

Spatially constrained clustering, also called regionalisation, have been proposed to derive spatially contiguous clusters.

One such method is the max-p problem. It aims to generate a maximum number of clusters conditioning that:

1. Each cluster is spatially contiguous. In other words, each cluster is a 'region';
2. The sum of a variable (e.g. population or number of spatial units) of each cluster is above a predefined threshold value. For example, the accumulative population of each cluster is above 10% of the total population, or the number of spatial units in each cluster is above 20% of the total spatial units.

Note that the number of clusters generated by the max-p method is not predefined; it is automatically determined by the algorithm.

This section is based on this [tutorial](https://pysal.org/spopt/notebooks/maxp.html).

Preparation:

In [ ]:
plt.rcParams["figure.figsize"] = [12, 8]
warnings.filterwarnings("ignore")

RANDOM_SEED = 123456

As the max-p problem is NP-hard and the algorithm is very slow on a dataset with hundreds of spatial units, we will use this algorithm on the MSOAs in the Camden borough.

Note that the field of MSOA11NM contains the borough name.

In [ ]:
ppd.MSOA11NM.str.contains('Camden')

In [ ]:
camden_msoa = ppd[ppd.MSOA11NM.str.contains('Camden')]
camden_normed = normed.iloc[ppd.MSOA11NM.str.contains('Camden').tolist()]
camden_msoa.shape

Here we aim to cluster the MSOAs into the maximum number of regions such that each region has at least 20% MOSAs and the homogeneity in several variables is maximised.

We first define the variables in the dataframe that will be used to measure regional homogeneity:

In [ ]:
attrs_name = camden_normed.columns.values.tolist()
attrs_name

Next, we specify a number of parameters that will serve as input to the max-p model.

A spatial weights object expresses the spatial connectivity of the MSOAs:

In [ ]:
w = libpysal.weights.Queen.from_dataframe(camden_msoa)

Then we specify the minimum number of MSOAs contained by each region (threshold):

In [ ]:
threshold = 0.2 * camden_normed.shape[0]
print(threshold)
camden_normed_copy = camden_normed.copy()
camden_normed_copy["count"] = 1
threshold_name = "count"

The max-p model can then be instantiated and solved. Note that this cell may take quite a long time to run. It takes 36.9 seconds on a 24G Windows 10 desktop.

In [ ]:
%%time
np.random.seed(RANDOM_SEED)
model = MaxP(camden_normed_copy, w, attrs_name, threshold_name, threshold, top_n=3)
model.solve()

In [ ]:
camden_msoa['cluster_max_p'] = model.labels_
camden_msoa.plot(column='cluster_max_p', categorical=True, legend=True, figsize=(12,8), cmap='Paired');

## Wrap-up

Now that you've practiced four clustering methods, including DBSCAN, Kmeans, hierarchical, and max-p.

If you want to understand the computational complexity of each approach, you can record the time each approach takes to run. For this purpose, you should explore how to use the `time.time()` function to measure how long the `.fit()` function takes to complete. You can also put the `%%time` on the top of a cell to measure the computing time of this cell.

What's the next steps?

- Add a lot more comments to the code to ensure that really have understood what is going on, and then save your own copy of this notebook in a Github repo.
- Try playing with some of the methods and parameters (e.g. replacing RobustScaler with MixMaxScaler, changing the number of clusters in hierarchical clustering) and seeing how your results change.
- Try outputting additional plots that will help you interpret your clustering results (e.g. what is the makeup of cluster 1? Or 6? What has it picked up? What names would I give these clsuters?).

## Credits!

### Acknowledgements:

This workshop is largely based on a [notebook](https://github.com/jreades/i2p/blob/master/lectures/10.2-Clustering-Housing.ipynb) developed by [Jon Reades](https://github.com/jreades). 

### References and extensions:

#### Spatially-constrained clustering:
- [Rgionalization algorithms](https://darribas.org/gds_course/content/bG/lab_G.html#regionalization-algorithms)
- [Max-p-regions algorithm](https://region.readthedocs.io/en/latest/users/max-p-regions/index.html)

#### In-depth notebooks on kmeans
- [In-depth Kmeans](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.11-K-Means.ipynb): part of the 'Python Data Science Handbook'. It illustrates the pros and cons of Kmeans with codes and examples and discusses using Kmeans for non-linear boundaries. HIGHLY recommended if you want to deep dive into Kmeans.